### 基于LangChain框架使用函数调用

In [30]:
!pip install langchain openai --upgrade --quiet

<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/function_call/function-call-based-on-langchain.ipynb" target="_parent">Open In Colab</a>

In [31]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
# from dotenv import load_dotenv
# load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'google.colab'

In [29]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import YouTubeSearchTool
from langchain_core.utils.function_calling import convert_to_openai_function

In [ ]:
model = 'gpt-3.5-turbo-0613'
llm = ChatOpenAI(model=model)

#### LangChain工具(Tool)的OpenAI函数调用能力

### 示例一：调用LangChain提供的工具

In [ ]:
tools = [YouTubeSearchTool()]
functions = [convert_to_openai_function(t) for t in tools]
functions

In [ ]:
message = llm.invoke([HumanMessage(content='search videos in the topic of OpenAI on Youtube')], functions=functions)
message

### 示例二：调用自定义的工具

In [ ]:
function_descriptions = [
    {
        "name": "remove_word_from_string",
        "description": "Remove a word from a string by given its index",
        "parameters": {
            "type": "object",
            "properties": {
                "string": {
                    "type": "string",
                    "description": "The original string to be processed",
                },
                "index": {
                    "type": "integer",
                    "description": "The index of the word to be removed"
                },
            },
            "required": [
                "string",
                "index"
            ],
        },
    },
    {
        "name": "send_message_by_email",
        "description": "Send an email with the text message to a recipient",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The email address of the recipient",
                },
                "message": {
                    "type": "string",
                    "description": "The message of the email content",
                }
            },
            "required": [
                "recipient",
                "message"
            ],
        },
    }
]

In [ ]:
question = """
I have a string as follows:

black yellow red blue green

Please do the following 2 operations on it:
1. Remove the third word in the string
2. Send the updated string to Alex via email alex@xyz.com
"""

In [ ]:
import inspect

def get_function_parameter_names(function):
    """
    获取指定函数的参数名列表。
    
    参数:
    function - 需要获取参数名的函数对象。
    
    返回值:
    如果函数参数有效，则返回一个包含所有参数名的列表；否则返回None。
    """
    if function is not None and inspect.isfunction(function):
        # 如果传入的对象是函数，获取其参数名
        parameter_names = inspect.signature(function).parameters.keys()
        return list(parameter_names)
    else:
        # 如果传入的对象不是函数，返回None
        return None


In [ ]:
def remove_word_from_string(string, index):
    words = string.split()

    if 0 <= index < len(words):
        del words[index]

        return ' '.join(words)
    else:
        return string

def send_message_by_email(recipient, message):
    print(f'Sending {message} to {recipient}')
    return f'Just sent email to {recipient}'

In [ ]:
first_response = llm.invoke([HumanMessage(content=question)], functions=function_descriptions)
first_response

In [ ]:
# Get function name, and its arguments
import json

function_name = first_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(first_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
    parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)
returned_value

In [ ]:
second_response = llm.invoke(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ],
    functions=function_descriptions
)
second_response

In [ ]:
# Again get function name, and its arguments

function_name = second_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(second_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)
returned_value

In [ ]:
third_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ], functions=function_descriptions
)
third_response